In [2]:
from pycocotools.coco import COCO
import layoutparser as lp
import random
import cv2
import os
import openpyxl

In [3]:
os.getcwd()

'/opt/Python_Code'

In [4]:
os.chdir("..")

In [7]:
pdfs = []
for num in range(2008,2022):
    year = str(num)
    files = os.listdir("Meeting_Minutes/" + year)
    for file in files:
        full = "Meeting_Minutes/" + year + '/' + file
        pdfs.append(full)
    print(num)

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [8]:
model = lp.Detectron2LayoutModel(
    config_path = "layout-model-training/outputs/prima/fast_rcnn_R_50_FPN_3x/config.yaml",
    model_path = "layout-model-training/outputs/prima/fast_rcnn_R_50_FPN_3x/model_final.pth",
    extra_config = ["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # <-- Only output high accuracy preds
)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from collections import defaultdict
pdf_bibs = defaultdict(list)
pdf_dates = list()

In [13]:
for meeting in pdfs:
    file = meeting
    pdf_tokens, pdf_images = lp.load_pdf(file, load_images=True)
    for page_index in range(len(pdf_images)): #Reference Section 
        bib_items = model.detect(pdf_images[page_index])
        for bib_item in bib_items:
            bib_tokens = pdf_tokens[page_index].filter_by(bib_item, center=True)
            bib_text = " ".join(bib_tokens.get_texts())
            pdf_bibs[page_index].append(bib_text)
            pdf_dates.append(meeting)
    print(meeting)

Meeting_Minutes/2008/2008-01-08; City Council (3).pdf
Meeting_Minutes/2008/2008-01-08; City Council.pdf
Meeting_Minutes/2008/2008-01-15; City Council (3).pdf
Meeting_Minutes/2008/2008-01-15; City Council.pdf
Meeting_Minutes/2008/2008-01-16; City Council.pdf
Meeting_Minutes/2008/2008-01-22; City Council (3).pdf
Meeting_Minutes/2008/2008-01-22; City Council.pdf
Meeting_Minutes/2008/2008-01-28; City Council.pdf
Meeting_Minutes/2008/2008-02-05; City Council (3).pdf
Meeting_Minutes/2008/2008-02-05; City Council.pdf
Meeting_Minutes/2008/2008-02-12; City Council (3).pdf
Meeting_Minutes/2008/2008-02-12; City Council.pdf
Meeting_Minutes/2008/2008-02-19; City Council (4).pdf
Meeting_Minutes/2008/2008-02-19; City Council.pdf
Meeting_Minutes/2008/2008-02-20; City Council.pdf
Meeting_Minutes/2008/2008-02-26; City Council (3).pdf
Meeting_Minutes/2008/2008-02-26; City Council.pdf
Meeting_Minutes/2008/2008-03-04; City Council.pdf
Meeting_Minutes/2008/2008-03-06; City Council.pdf
Meeting_Minutes/2008/2

In [14]:
import pandas as pd 

testdf = (pd
 .DataFrame(tuple(pdf_bibs.items()), 
             columns=['page_index', 'bib-item'])
 .explode('bib-item')
 .reset_index(drop=True)
)

testdf["meeting"] = pdf_dates

In [16]:
testdf.to_excel("Data_Output/carlsbad_raw.xlsx")